In [1]:
import multiprocessing as mp
import itertools as it
import functools as ft
import pickle
import sys
import numpy as np
import pandas as pd
import time
import sklearn.preprocessing as pre
import scipy.sparse as sp

In [2]:
import torch

In [3]:
dat = pd.read_csv('./cord_blood_kinases.csv', sep=',', header=0, index_col=0)
dat.head()

/Users/isrobson/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,barcode,symbol
1,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,STK4
2,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,RIOK3
3,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,PDK2
4,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,MAPK7
5,MantonCB1_HiSeq_1-AAACCTGCACAGACAG-1,CDK10


In [4]:
cores = max(mp.cpu_count()-1, 1)
total_cells = dat['barcode'].nunique()
cell_id_dict = {y:x for x,y in enumerate(dat['symbol'].unique())}
dat['symbol'] = dat['symbol'].map(cell_id_dict)
cells = dat['symbol'].unique()
dat.sort_values(by='symbol',inplace=True)

# create slices to know how to sum sparse matrix
slices = pd.DataFrame(dat.groupby('symbol').count().cumsum())
slices.columns = ['upper']
slices['lower'] = [0] + list(slices['upper'])[0:-1]

# 
lab_enc = pre.LabelEncoder()
one_hot = pre.OneHotEncoder(categories='auto')
oe_data = one_hot.fit_transform((lab_enc.fit_transform(dat['barcode'].values)).reshape(-1,1))

def get_cell_sets(row, oe_csr):
    return oe_csr[row['lower']:row['upper']].sum(axis=0)
get_cell_partial = ft.partial(get_cell_sets, oe_csr=oe_data)

cell_sets = slices.apply(get_cell_partial, axis=1)

In [52]:
b = ss.csr_matrix(np.stack(cell_sets.values))

In [60]:
b.shape

(550, 273842)

In [62]:
b.dot(b.transpose()) > 100

<550x550 sparse matrix of type '<class 'numpy.bool_'>'
	with 109883 stored elements in Compressed Sparse Row format>

In [56]:
b.transpose().shape

(273842, 550)

In [10]:
b.shape[0]

550

In [7]:
c = torch.unsqueeze(b.t(), dim =2)
c.shape

torch.Size([273842, 550, 1])

In [17]:
a_sparse = ss.coo_matrix(np.stack(cell_sets.values))

In [21]:
(a_sparse.transpose()).shape

(273842, 550)

In [20]:
a_sparse.shape

(550, 273842)

In [19]:
a_sparse.multiply(other=a_sparse.transpose())

ValueError: inconsistent shapes

In [23]:
values = a_sparse.data
indices = np.vstack((a_sparse.row, a_sparse.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = a_sparse.shape

In [26]:
c = torch.sparse.ShortTensor(i, v, shape)

In [13]:
res = []
for i in range(c.shape[1]):
    res.append(torch.matmul(b, c[:,i,:]))

KeyboardInterrupt: 

In [46]:
d.shape

torch.Size([1, 550, 273842])

In [ ]:
torch.matmul(d,c).shape